# Creating the Initial Commitment Transaction

In this section, we'll build a lightning channel initial commitment transaction from scratch using Python. We'll walk through each part of the transaction — how it's constructed and signed. The process will be tested using Bitcoin Core in regtest mode.

## Setup

For this notebook, we’ll use the basepoints derivated in the `chapter 0 - lightning node keys derivation`, and the funding transaction created in the `chapter 1 - channel funding transaction`.

In [1]:
# run notebook
%run "/home/pins-dev/Projects/Taproot-Lightning-Channels-Workshop/chapter 0 - lightning node keys derivation/lightning node keys derivation.ipynb"
%run "/home/pins-dev/Projects/Taproot-Lightning-Channels-Workshop/chapter 1 - channel funding transaction/funding-transaction.ipynb"

Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Revocation Basepoint Private Key: c17ac3952ca414190074d1e59ea03fbae253196173908dc8b131af6bd2cc8161
Alice Revocation Basepoint Public Key: 03649c4f865bec74b0a186deef4defad51cfdc141443e38074ea05a7835a953a49
Alice HTLC Basepoint Private Key: 763ae49a20e6668c88602c782716dd83ba6c4cc0333b38810e2bcd7b22c871ac
Alice HTLC Basepoint Public Key: 02816fde4150e4dfcac94eff0b821448fb70f57a56148ba2206cd9b2fd0cc20bdf
Alice Payment Basepoint Private Key: 72d8c12971b58076a1f27eb7938ca442f0b210762b23637443ac2e99dac352a6
Alice Payment Basepoint Public Key: 025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Alice Delayed Payment Basepoint Private Key: 7cafce00c54e7241894dcc7c3beaca29dd354139fdb6182198d6c5f1063bfe8d
Alice Delayed Payment Basepoint Public Key: 034aa35219136bb238e072341b20a4bf8fb44a83cdb73dd2bd9

At this point, no HTLCs have been added yet, which makes the initial commitment transaction simpler. Only after Alice has a fully signed initial commitment transaction she will broadcast the funding transaction, this serves as a guarantee for Alice (the funder) that she can reclaim her sats back if anything goes wrong.

### The Unsigned Transaction Inputs

In [2]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(funding_channel_txid))[::-1]
funding_channel_index = 0
index = funding_channel_index.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# sequence: upper 8 bits are 0x80, lower 24 bits are the upper 24 bits of the obscured commitment number
# Commitment number on the opening channel 
commitment_number = 0
# obscured commitment number is result of xor operation 
commitment_number_obscured = lower48_to_obscure ^ commitment_number
# Extract the upper 24 bits of the obscured commitment number
upper_24_bits = (commitment_number_obscured >> 24) & 0xFFFFFF
# Combine the upper 8 bits (0x80) with the lower 24 bits (upper 24 of obscured number)
sequence = (0x80 << 24) | upper_24_bits
# Convert to bytes (byte, big-endian)
sequence = sequence.to_bytes(4, byteorder='big')
# Convert to little-endian 
sequence = sequence[::-1]

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

In [ ]:
# create an object for Alice’s first per-commitment
alice_per_commitment = PerCommitmentKey(alice_per_commitment_seed, 0xFFFFFFFFFFFF)

print(f"Alice First Per Commitment Point: {alice_per_commitment.get_compressed().hex()}")